In [1]:
from pcap_util import PcapReader
import struct
import socket
from tcp_util import Flow




In [2]:
class A:
    pass

a = A()
a.src = '119.215.241.99'
a.sport = '443'
a.dst = '171.199.232.237'
a.dport = '60215'
f1 = Flow({'IP':a, 'TCP':a})

b = A()
b.src = '171.199.232.237'
b.sport = '60215'
b.dst = '119.215.241.99'
b.dport = '443'
f2 = Flow({'IP':b, 'TCP':b})

print(f1, hash(f1), f1.src.ip, f1.dst.ip, f1.src.port, f1.dst.port)
print(f2, hash(f2), f2.src.ip, f2.dst.ip)


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
 119.215.241.99:443   ==> 171.199.232.237:60215 3677248384076997205 2010640739 2882005229 443 60215
171.199.232.237:60215 ==>  119.215.241.99:443   3677248384076997205 2882005229 2010640739


In [6]:
FLAGS = {
	"F":0x1, "S":0x2, "R":0x4, "P":0x8, "A":0x10, "U":0x20, "E":0x40, "C":0x80,
#	0x1:"F", 0x2:"S", 0x4:"R", 0x8:"P", 0x10:"A", 0x20:"U", 0x40:"E", 0x80:"C"
}

FLAGS_ORDER = "SAFCR"

CLIENT_STATES = {
    'CLOSED': {
        'S': 'SYN_SENT',
    },
    'SYN_SENT': {
        'SA': 'ESTABLISHED',
        # deal with timeout to go back to CLOSED
    },
    'ESTABLISHED': {
        'F': 'FIN_WAIT_1', # (FIN comes from client)
    },
    'FIN_WAIT_1' : {
        'A': 'FIN_WAIT_2', # passive close, client sends nothing
        'F': 'CLOSING', # simultaneous close, client sends ACK
        'FA': 'TIME_WAIT', # FIN and ACK in one packet, same as simultaneous close, client sends ACK
    },
    'CLOSING': {
        'A': 'TIME_WAIT', # client sends nothing
    },
    'FIN_WAIT_2': {
        'F': 'TIME_WAIT', # could wait for timeout, but all transitions to this state lead to CLOSED
    },
}

SERVER_STATES = {
    'LISTEN': {
        'S' : 'SYN_RCVD', # SYN
    },
    'SYN_RCVD': {
        'A': 'ESTABLISHED', # ACK
        'R': 'LISTEN', # RESET
        'F': 'FIN_WAIT_1' # active close
    },
    'ESTABLISHED': {
        'F': 'CLOSE_WAIT', # server sends ACK
    },
    'CLOSE_WAIT': {
        'F': 'LAST_ACK', # (FIN comes from server) server sends FIN
    },
    'LAST_ACK': {
        'A': 'CLOSED',
    }
}


In [3]:
is_syn_pkt = lambda pkt: 'TCP' in pkt and pkt['TCP'].flags == FLAGS['S']

class TCPSession:
	def __init__(self, pkt):
		if not 'TCP' in pkt:
			raise Exception("Not a TCP Packet")
		if not is_syn_pkt(pkt):
			raise Exception("Not valid SYN")

		self.flow = Flow(pkt) # client => server
#		self.packets = []
		self.count = 0

		# 0 is now, 1 is the future Flags
		self.server_state = "LISTEN"
		self.client_state = "CLOSED"

		self.handle_packet(pkt)
#       self.server_close_time = -1.0
#       self.client_close_time = -1.0
#       self.fin_wait_time = -1.0

	@property
	def client(self):
		return self.flow.src
	@property
	def server(self):
		return self.flow.dst

	def add(self, pkt):
		if not 'TCP' in pkt:
			raise Exception("Not a TCP Packet")

		# determine in what context we are handling this packet
#        flow = Flow(pkt)
#        if flow.forward != self.flow and flow.reverse != self.flow:
#            raise Exception("Not a valid packet for this model")

		self.handle_packet(pkt)
#		if flow.dst == self.server:
#			v =  self.add_client_pkt(pkt)
#			if self.is_fin_wait():
#				self.fin_wait_time = pkt.time
#			return v
#		else:
#			v = self.add_server_pkt(pkt)
#			if self.is_fin_wait():
#				self.fin_wait_time = pkt.time
#			return v
#		raise Exception("Not a valid packet for this model")

	def handle_packet(self, pkt):
#		flags = pkt['TCP'].flags
#		flags = ''.join(f for f,x in FLAGS.items() if x & flags)
#		flags = ''.join(f for f in FLAGS_ORDER if f in flags)

		flow = Flow(pkt)

		if flow.dst == self.server:
			state = self.handle_client_packet(pkt)
		else:
			state = self.handle_server_packet(pkt)

		if self.is_fin_wait():
			self.fin_wait_time = float(pkt.time)

#		self.packets.append(bytes(pkt))

		return state
#		self.client_state = CLIENT_STATES[self.client_state].get(flags) or self.client_state
#		self.server_state = SERVER_STATES[self.server_state].get(flags) or self.server_state
		

	def active_close(self):
		return (self.client_state == self.server_state and self.server_state == "CLOSED")

	def passive_close(self):
		return (self.client_state == "LAST_ACK" and self.server_state == "CLOSE_WAIT")

	def is_closed(self):
		return self.passive_close() or self.active_close()

	def is_fin_wait(self):
		return self.client_state.find("FIN_WAIT") > -1 or self.server_state.find("FIN_WAIT") > -1

	def build_flags(self, sflags):
		return sum([FLAGS[i] for i in sflags])

	def handle_client_packet(self, pkt):
		flags = pkt['TCP'].flags

		if flags == self.build_flags("R") or flags == self.build_flags('RA'):
			self.client_state = "CLOSED"

		elif self.client_state == 'CLOSED':
			if flags == self.build_flags("S"):
				self.client_state = "SYN_SENT"

		elif self.client_state == "SYN_SENT":
			if flags == self.build_flags("A"):
				self.client_state = "ESTABLISHED"

		elif self.client_state == "ESTABLISHED":
			if flags & self.build_flags("F"):
				self.client_state = "FIN_WAIT_1"
			if flags & self.build_flags('A') and self.server_state == 'FIN_WAIT_1':
				self.client_state = 'CLOSE_WAIT'
			if flags == 0 or flags == self.build_flags('A'):
				return self.is_closed()

		elif self.client_state == "FIN_WAIT_1":
			if flags == self.build_flags("A"):
				self.client_state = "CLOSED"

		elif self.client_state == "ESTABLISHED" and self.server_state == "CLOSE_WAIT" and flags & self.build_flags("A") > 0:
			self.client_state = "CLOSED"

		if self.server_state == "FIN_WAIT_1" and self.client_state == "CLOSED" and flags == self.build_flags("A"):
			self.server_state = "CLOSED"


		return self.is_closed()

	def handle_server_packet(self, pkt):
		flags = pkt['TCP'].flags
		server_got_closed = False
		client_got_closed = False

		if flags == self.build_flags("R"):
			self.client_state = "CLOSED"
			self.server_state = "CLOSED"
			server_got_closed = True
			client_got_closed = True

		elif flags == self.build_flags("RA"):
			self.client_state = "CLOSED"
			self.server_state = "CLOSED"
			server_got_closed = True
			client_got_closed = True

		elif flags == self.build_flags("S"):
			self.server_state = "SYN_SENT"
		elif self.server_state == "LISTEN" and\
			flags == self.build_flags("SA"):
			self.server_state = "SYN_RCVD"

		elif self.server_state == "ESTABLISHED" and\
			flags == self.build_flags("FA"):
			self.server_state = "FIN_WAIT_1"

		elif self.server_state == "FIN_WAIT_1" and\
			flags == self.build_flags("A"):
			self.server_state = "CLOSED"
			server_got_closed = True


		elif self.server_state == "SYN_RCVD" and\
			flags == self.build_flags("F"):
			self.server_state = "FIN_WAIT_1"

		elif self.server_state == "FIN_WAIT_1" and\
			flags == self.build_flags("FA"):
			self.server_state = "CLOSED"

		elif self.server_state == "SYN_RCVD" and\
			flags == self.build_flags("A"):
			self.server_state = "ESTABLISHED"

		elif self.server_state == "ESTABLISHED" and\
			flags & self.build_flags("F") > 0:
			self.server_state = "CLOSE_WAIT"

		elif self.client_state == "FIN_WAIT_1" and\
			flags == self.build_flags("FA"):
			self.server_state = "CLOSED"
			server_got_closed = True

		elif self.client_state == "CLOSED" and\
			flags == self.build_flags("A"):
			self.server_state = "CLOSED"
			server_got_closed = True

		if self.client_state == "FIN_WAIT_1" and\
			self.server_state == "CLOSED" and\
			flags == self.build_flags("A"):
			self.client_state = "CLOSED"
			client_got_closed = True

		if client_got_closed:
			self.client_close_timed = float(pkt.time)
		if server_got_closed:
			self.server_close_timed = float(pkt.time)

		return self.is_closed()

In [7]:

#reader = scapy.all.rdpcap('../data/pcap/eq1.pcap', count=100000)
reader = PcapReader('../data/pcap/abs.tcp.nano.pcap')
sessions = {}
closed = []

for pkt in reader:
    if 'IP' not in pkt or 'TCP' not in pkt:
        continue

    flow = Flow(pkt)

    if flow not in sessions and is_syn_pkt(pkt):
        sessions[flow] = TCPSession(pkt)
    elif flow in sessions:
        sessions[flow].add(pkt)

        if sessions[flow].is_closed():
            closed.append(sessions[flow])
            del sessions[flow]

for session in closed:
    print(session.flow, len(session.packets))

  161.55.36.102:25393 ==>  51.226.116.243:23    2
 161.53.113.220:62117 ==>  119.18.106.200:5001  2
 171.222.162.83:65025 ==> 117.170.109.153:445   2
   25.20.169.74:53640 ==>   71.101.120.44:60000 2
  161.39.221.43:54778 ==>   71.101.120.95:60000 2
    55.4.226.46:38356 ==>   71.101.120.41:60000 2
   25.20.169.74:43020 ==>  71.101.121.240:60000 2
    55.4.226.46:36004 ==>  71.101.116.183:60000 2
  161.39.221.43:45034 ==>   71.101.126.36:60000 2
   25.23.184.48:56928 ==>   71.101.123.72:60000 2
   25.23.184.48:57826 ==>  71.101.125.196:60000 2
   25.20.169.74:51066 ==>  71.101.124.171:60000 2
  161.39.221.43:53504 ==>  71.101.118.251:60000 2
   25.23.184.87:34204 ==>   71.101.122.25:60000 2
   25.20.169.74:55970 ==>  71.101.121.151:60000 2
   25.23.184.48:49126 ==>  71.101.123.207:60000 2
   25.23.184.48:52458 ==>   71.101.121.48:60000 2
  161.39.221.43:54546 ==>  71.101.123.107:60000 2
  161.39.221.43:53720 ==>   71.101.124.82:60000 2
   25.23.184.48:41666 ==>  71.101.122.148:60000 2


In [9]:
for session in sorted(closed, key=lambda s: len(s.packets), reverse=True):
    print(session.flow, len(session.packets))

  162.67.163.69:57013 ==>   93.191.29.101:80    14
  180.222.92.23:20989 ==>    178.42.92.19:443   10
170.250.239.192:34968 ==>    178.42.92.19:443   10
 123.138.57.129:42227 ==>  71.113.100.141:443   10
  170.238.72.61:37819 ==>    178.42.92.21:443   10
 182.16.180.196:33155 ==>    178.42.92.21:443   10
  161.5.165.130:49869 ==>  199.52.215.159:443   10
 123.138.57.186:45075 ==>    35.88.144.58:443   10
  175.73.60.180:40351 ==>   207.11.248.12:443   10
  136.87.68.123:38706 ==>    178.42.92.44:443   10
173.244.147.245:60306 ==> 181.187.115.146:80    9
 162.190.34.139:14393 ==>   69.27.148.209:443   9
  146.55.105.97:7164  ==>   34.195.166.72:80    8
   136.185.4.72:60055 ==>   119.17.41.200:443   8
 182.16.246.159:44242 ==>  145.184.77.111:443   8
  146.55.105.94:25888 ==>   34.201.223.25:80    8
  161.5.162.248:43120 ==>    52.98.177.10:80    8
 146.55.105.103:38566 ==>  55.101.199.163:80    7
 146.55.105.107:37408 ==>  51.143.135.234:80    7
  146.55.105.92:41638 ==>   34.201.223.2

In [53]:
import glob

closed = []

#for fn in glob.glob('../data/pcap/sessions_caida_small/*.pcap'):
for fn in glob.glob('test/*.pcap'):
	session = None

	for pkt in scapy.all.rdpcap(fn):
		flow = Flow(pkt)
		flags = pkt['TCP'].flags
		flags = ''.join(f for f,x in FLAGS.items() if x & flags)
#		flags = ''.join(f for f in FLAGS_ORDER if f in flags)
		if session is None:
			session = TCPSession(pkt)
		else:
			session.add(pkt)

#		if flow.src != session.flow.src:
		print(flow, flags)

#		print(session.client_state, session.server_state)

	closed.append(session)
#	break


   192.168.0.11:52350 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11:52350 RA
   192.168.0.11:52362 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11:52362 RA
   192.168.0.11:52370 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11:52370 RA
   192.168.0.11:52366 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11:52366 RA
   192.168.0.11:52365 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11:52365 RA
   192.168.0.11:52369 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11:52369 RA
   192.168.0.11:52361 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11:52361 RA
   192.168.0.11:52349 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11:52349 RA
   192.168.0.11:52372 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11:52372 RA
   192.168.0.11:52356 ==>    192.168.0.12:666   S
   192.168.0.12:666   ==>    192.168.0.11